# Utilities

In [5]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd 
from sklearn.preprocessing import StandardScaler

Calculate box boundries

In [2]:
def calculate_box(image, results):
    image_width, image_height = image.shape[1] , image.shape[0]
    landmark_array = np.empty((0,2),int)
    for landmarks in results: 
        for i, landmark in enumerate(landmarks.landmark):
            landmark_x = min(int(landmark.x * image_width), image_width -1)
            landmark_y = min(int(landmark.y * image_height), image_height -1)
            landmark_point = [np.array((landmark_x, landmark_y))]
            landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x,y,w,h = cv2.boundingRect(landmark_array)
    return [x,y,x+w,y+h]

Draw box

In [13]:
def draw_box(draw_box, image, box):
    if draw_box:
        #draw rectangle image , point 1, point 2, color, thickness
        cv2.rectangle(image, (box[0],box[1]),(box[2],box[3]),( 66, 194,245),1)
    return image

Get landmarks data

In [14]:
def get_landmarks_snapshot(results):
    data = []
    columns = []
    for landmarks in results: 
        for i, landmark in enumerate(landmarks.landmark):
            columns.append(f"Joint {i +1} - X" )
            columns.append(f"Joint {i +1} - Y" )
            landmark_x = landmark.x
            landmark_y = landmark.y
            data.append(landmark_x)
            data.append(landmark_y)
        df = pd.DataFrame(columns=columns)
        df.loc[len(df.index)] = data
    return df

Get landmarks angles

In [7]:
def get_landmarks_angles(results, joint_list):
    data = []
    for hand in results:
        for joint in joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # First coord
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # Second coord
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # Third coord

            radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)

            if angle > 180.0:
                angle = 360-angle
            data.append(angle)
    data = np.array(data)
    return data

Get scaler

In [ ]:
def get_scaler(data):
    scaler - StandardScaler()
    scaler.fit(data)
    return scaler

Create labels

In [16]:
import json

labels = [
    {'label':'Open', 'id':0},
    {'label':'Closed', 'id':1},
    {'label': 'OK', 'id':2},
    {'label': 'Peace', 'id':3},
]

with open('labels.txt', 'w') as f:
     f.write(json.dumps(labels))

Finger angles

In [23]:
def draw_finger_angles(image, results, joint_list):
    
    # Loop through hands
    for hand in results.multi_hand_landmarks:
        #Loop through joint sets 
        for joint in joint_list:
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # First coord
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # Second coord
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # Third coord
            
            radians = np.arctan2(c[1] - b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)
            
            if angle > 180.0:
                angle = 360-angle
                
            cv2.putText(image, str(round(angle, 2)), tuple(np.multiply(b, [640, 480]).astype(int)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.25, (255, 255, 255), 1, cv2.LINE_AA)
    return image